In [1]:
import urllib.request
from bs4 import BeautifulSoup as soup
import requests
import re
import pandas as pd
import numpy as np

link = 'https://rally-base.com/2024/eko-acropolis-rally-2024/?ssId='
startat, no_ss=8436, int(15) # starting number of url, count of Special Stages
canceled = []

stages = [number for number in range(no_ss)]

In [2]:
stages_24 = []
overall_24 = []

In [3]:
if canceled:
    for j in canceled: stages.remove(j-1)

In [4]:
for ss in stages:
    val= startat + ss
    ss_a = str(val)
    my_url11 = link + ss_a
    #print(ss, val, ss_a, "\n", my_url11)
    
    req = urllib.request.Request(my_url11, headers={'User-Agent': 'Mozilla/5.0'})
    uClient11 = urllib.request.urlopen(req)
    page_html11 = uClient11.read()
    uClient11.close()
    data = pd.read_html(page_html11)[2]
   
    data.columns = ['Pos.', 'No', 'Prev_Pos.', 'Crew', 'Gr-Pos','time_and_pen', 'total_time', 'Diff']
    data = data[1:]
    data['ss']=ss+1

    overall = pd.read_html(page_html11)[2]
   
    overall.columns = overall.iloc[0]
    overall = overall[1:]
    overall['ss']=ss+1

    #print(overall.columns)
    
    equal = '-' in data['Pos.'].unique()
    if equal:
        data['Pos.'] = data['Pos.'].replace('-', method='ffill')
    

    stages_24.append(data)
    overall_24.append(overall)


In [5]:
#rally stages Separately classification
rally2024_overall = pd.concat(stages_24, axis=0)
rally2024_overall['Pos.'] = rally2024_overall['Pos.'].astype(str).astype(int)
rally2024_overall['No'] = rally2024_overall['No'].astype(str).astype(int)
rally2024_overall[['Group', 'Group_Pos']] = rally2024_overall['Gr-Pos'].str.split(' ', expand=True)
rally2024_overall[['stages_time', 'Penalty']] = rally2024_overall['time_and_pen'].str.split(' ', expand=True)
rally2024_overall[['Diff_first', 'Diff_Prev']] = rally2024_overall['Diff'].str.split(' ', expand=True)
rally2024_overall['Penalty'] = rally2024_overall['Penalty'].replace({None: 0})
rally2024_overall = rally2024_overall.drop(['Gr-Pos', 'time_and_pen', 'Diff'],axis=1)
rally2024_overall['Crew'] = rally2024_overall['Crew'].str[:12]
rally2024_overall.to_csv('rally2024_overallStages.csv', index=False)

In [6]:
rally_overall_time_temp=rally2024_overall.drop(['Prev_Pos.','Pos.', 'No','stages_time', 'Penalty', 'Diff_first', 'Diff_Prev', 'Group_Pos'],axis=1)
rally_overall_position_temp=rally2024_overall.drop(['Prev_Pos.','total_time', 'No','stages_time', 'Penalty', 'Diff_first', 'Diff_Prev'],axis=1)

In [8]:
#prepare for export
rally2024_overall_Positions = rally_overall_position_temp.drop(['Group_Pos'], axis=1)
rally2024_overall_Positions_view = rally2024_overall_Positions.set_index(['Crew','ss', 'Group'], drop=True).unstack('ss')
rally2024_Group_Positions = rally_overall_position_temp.drop(['Pos.'], axis=1)
rally2024_Group_Positions_view = rally2024_Group_Positions.set_index(['Crew','ss', 'Group'], drop=True).unstack('ss')
rally2024_overall_time_view = rally_overall_time_temp.set_index(['Crew','ss', 'Group'], drop=True).unstack('ss')
rally2024_Group_Positions_view

Group_Pos                                                  \
ss                        1   2   3    4    5    6    7    8    9    10   11   
Crew         Group                                                             
Al-Wahaibi   RC2          16  16  15   15   20   20   17   17   17   17   25   
Amoutzas  G. RC2          33  33  33   31   31   30   29   29   29   29   27   
Athanassoula RC2          32  32  32  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
Blach  Rober RC2          15  14  14   16   14   14   14   14   13   13   12   
Borodin  Pet RC3           9   9   8    9    7    7    6    6    5    5    5   
...                      ...  ..  ..  ...  ...  ...  ...  ...  ...  ...  ...   
Villanueva   RC2          12  12  11   11   15   12   13   13   12   12   15   
Virves  Robe RC2           9   9   5    4    4    1    1    1    2    2    2   
Zaldivar  F. RC2           8   7   7    6    5    4    3    4    4    4    6   
Çetinkaya  B RC2          27  27  25   24   21   21   19   19   22   25   24   
Šekuljica  S RC3          20  20  21   21   21   20  NaN  NaN  NaN  NaN  NaN   

                                        
ss                   12   13   14   15  
Crew         Group                      
Al-Wahaibi   RC2    NaN  NaN  NaN  NaN  
Amoutzas  G. RC2     23  NaN  NaN  NaN  
Athanassoula RC2    NaN  NaN  NaN  NaN  
Blach  Rober RC2     12   12  NaN  NaN  
Borodin  Pet RC3      4    4  NaN  NaN  
...                 ...  ...  ...  ...  
Villanueva   RC2    NaN  NaN  NaN  NaN  
Virves  Robe RC2      2    2    2    2  
Zaldivar  F. RC2      6    6    5    5  
Çetinkaya  B RC2     22   22   21   18  
Šekuljica  S RC3    NaN  NaN  NaN  NaN  

[70 rows x 15 columns]

In [9]:
rally2024_overall_Positions_view.to_csv('Positionoverall.csv')
rally2024_Group_Positions_view.to_csv('PositionGroup.csv')
rally2024_overall_time_view.to_csv('PositionoverallTime.csv')

In [37]:
rally_overall_time_temp

,Crew,total_time,ss,Group
1,Ogier S. /,00:16:33.2,1,RC1
2,Tänak Ott /,00:16:33.8,1,RC1
3,Fourmaux Ad,00:16:34.0,1,RC1
4,Katsuta T.,00:16:35.9,1,RC1
5,Sordo Dani,00:16:42.5,1,RC1
...,...,...,...,...
48,Gryazin Nik,05:09:21.9,15,RC2
49,Parellis D.,05:10:27.5,15,RC5
50,Cărăgui Eug,05:11:58.8,15,RC2
51,Plagos Ioan,05:33:25.7,15,RC4


In [6]:
rally2024_overall

,Pos.,No,Prev_Pos.,Crew,Gr-Pos,time_and_pen,total_time,Diff,ss,Group,Group_Pos,stages_time,Penalty,Diff_first,Diff_Prev
1,1,17,NaN,Ogier S. /,RC1 1,00:16:33.2,00:16:33.2,00:00:00.0 00:00:00.0,1,RC1,1,00:16:33.2,0,00:00:00.0,00:00:00.0
2,2,8,NaN,Tänak Ott /,RC1 2,00:16:33.8,00:16:33.8,00:00:00.6 00:00:00.6,1,RC1,2,00:16:33.8,0,00:00:00.6,00:00:00.6
3,3,16,NaN,Fourmaux Ad,RC1 3,00:16:34.0,00:16:34.0,00:00:00.8 00:00:00.2,1,RC1,3,00:16:34.0,0,00:00:00.8,00:00:00.2
4,4,18,NaN,Katsuta T.,RC1 4,00:16:35.9,00:16:35.9,00:00:02.7 00:00:01.9,1,RC1,4,00:16:35.9,0,00:00:02.7,00:00:01.9
5,5,6,NaN,Sordo Dani,RC1 5,00:16:42.5,00:16:42.5,00:00:09.3 00:00:06.6,1,RC1,5,00:16:42.5,0,00:00:09.3,00:00:06.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,48,23,51.0,Gryazin Nik,RC2 21,05:05:31.9 00:03:50.0,05:09:21.9,01:31:17.7 00:01:05.9,15,RC2,21,05:05:31.9,00:03:50.0,01:31:17.7,00:01:05.9
49,49,85,48.0,Parellis D.,RC5 1,05:10:27.5,05:10:27.5,01:32:23.3 00:01:05.6,15,RC5,1,05:10:27.5,0,01:32:23.3,00:01:05.6
50,50,77,52.0,Cărăgui Eug,RC2 22,05:11:48.8 00:00:10.0,05:11:58.8,01:33:54.6 00:01:31.3,15,RC2,22,05:11:48.8,00:00:10.0,01:33:54.6,00:01:31.3
51,51,82,53.0,Plagos Ioan,RC4 3,05:33:25.7,05:33:25.7,01:55:21.5 00:21:26.9,15,RC4,3,05:33:25.7,0,01:55:21.5,00:21:26.9
